# GLOSSARY CHATBOT

This code is an implementation of a simple chatbot using Python. The chatbot is designed to read information from a Word document, process user input, and provide relevant responses based on pre-defined answers. 

### Importing Important Libraries

In [1]:
#library to load the word document to the code.
from docx import Document
#library use SequenceMatcher to return list of the best "good enough" matches.
from difflib import get_close_matches
#to perform NLP tasks like intent recognition and entity extraction. 
import spacy
#loads pre-trained models from spaCy that are capable of processing natural language text for various NLP tasks
nlp = spacy.load("en_core_web_sm")

## Functions: 

This function used to extract data from tables in a Word document (.docx file) and convert it into a list of lists.

In [2]:
def read_table_from_docx(file_path):
    doc = Document(file_path)
    tables = doc.tables
    data = []
    for table in tables:
        for row in table.rows:
            row_data = [cell.text.strip() for cell in row.cells]
            data.append(row_data)
    return data

This function processes the (table_data), which is a list of lists containing table information, and generates a dictionary (chatbot_responses) that maps questions to their corresponding answers.

In [3]:
def generate_chatbot_responses(table_data):
    chatbot_responses = {}
    for row in table_data:
        if len(row) >= 2:
            question = row[0]
            answer = row[1]
            chatbot_responses[question.lower()] = answer
    return chatbot_responses

#### This function is used to find the keyword in the list of keywords that best matches the input user_input.

It measures the similarity between each word in the user_input and each keyword using the similarity_ratio() function, and then returns the keyword that has the highest similarity with any word in the user input.

In [4]:
def find_closest_match(user_input, keywords):
    user_input_lower = user_input.lower()
    words = user_input_lower.split()
    best_match = None
    max_similarity = 0

    for keyword in keywords:
        for word in words:
            similarity = similarity_ratio(word, keyword)
            if similarity > max_similarity:
                max_similarity = similarity
                best_match = keyword

    return best_match

#### This function calculates the similarity ratio between two sets of elements s1 and s2. 

It is used to measure the similarity between two sequences of elements, represented by the sets s1 and s2.

In [5]:
def similarity_ratio(s1, s2):
    s1_words = set(s1)
    s2_words = set(s2)
    intersection = s1_words.intersection(s2_words)
    return len(intersection) / (len(s1_words) + len(s2_words) - len(intersection))

This function is used to find the appropriate answer from the chatbot_responses dictionary based on the user's input user_input. The function searches for the user's input as a question in the keys of the chatbot_responses dictionary, and if a matching question is found, it returns the corresponding answer.

In [6]:
def find_answer_by_keyword(user_input, chatbot_responses):
    user_input_lower = user_input.lower()
    for question, answer in chatbot_responses.items():
        if question in user_input_lower:
            return answer
    return None

 This function is used to suggest an alternative question from the chatbot_responses dictionary based on the user's input user_input. If the user input does not exactly match any of the questions in the chatbot_responses dictionary, the function tries to find the closest matching question (keyword) and suggests it as an alternative.

In [7]:
def suggest_alternative(user_input, chatbot_responses):
    keywords = chatbot_responses.keys()
    closest_match = find_closest_match(user_input, keywords)
    return closest_match

 This main function allows the chatbot to interact with the user. It prompts the user with a greeting message, listens to the user's input, and provides responses based on the input and the chatbot's pre-defined responses.

In [8]:
def chat_with_bot(chatbot_responses):
    conversation_history = []
    print("Chatbot: Hi! How Can I Assest You Today?.")
    while True:
        user_input = input("You: ")
        user_input_lower = user_input.lower()
        if user_input_lower == "exit":
            print("NGMSA-CC: Goodbye!")
            break
        else:
            conversation_history.append(user_input)
            answer = find_answer_by_keyword(user_input_lower, chatbot_responses)
            if answer:
                print("NGMSA-CC:", answer)
            else:
                suggestion = suggest_alternative(user_input_lower, chatbot_responses)
                if suggestion:
                    print("NGMSA-CC: Did you mean:", suggestion, "?")
                    user_response = input("You: ")
                    if user_response.lower() == "yes" or user_response.lower() == suggestion :
                        conversation_history.append(user_response)
                        suggested_answer = chatbot_responses.get(suggestion)
                        if suggested_answer:
                            print("NGMSA-CC:", suggested_answer)
                        else:
                            print("NGMSA-CC: I'm sorry, I don't have an answer for that question.")
                    else:
                        print("NGMSA-CC: I'm sorry, I don't have an answer for that question.")
                else:
                    #function uses the spaCy library to processes user input and extracts the intent (main verb) and named entities (e.g., names, organizations) from the input.
                    intent, entities = process_user_input(user_input)
                    handle_intent(intent, entities, chatbot_responses, conversation_history)


This function to process the user's input using spaCy, an NLP (Natural Language Processing) library. 

In [9]:

def process_user_input(user_input):
    # Use spaCy to process user input and extract intent and entities
    doc = nlp(user_input)
    intent = None
    entities = {}
    for token in doc:
        if token.ent_type_:
            entities[token.ent_type_] = token.text
        if token.pos_ == "VERB" and intent is None:
            intent = token.text
    return intent, entities


This function responsible for handling user intents and generating responses based on the extracted intent and entities from the user's input. It customizes the chatbot's behavior based on the user's specific intent and named entities.

In [10]:

def handle_intent(intent, entities, chatbot_responses, conversation_history):
    # Implement custom logic to handle user intents and generate responses
    if intent == "ask_about":
        entity = entities.get("PERSON") or entities.get("ORG")
        if entity:
            # Check if a similar question is already asked in the conversation history
            similar_question = find_closest_match(entity.lower(), conversation_history)
            if similar_question:
                answer = find_answer_by_keyword(similar_question, chatbot_responses)
                if answer:
                    print("NGMSA-CC:", answer)
                else:
                    print("NGMSA-CC: I'm sorry, I don't have an answer for that question.")
            else:
                print("NGMSA-CC: I'm sorry, I don't have any information about", entity)
        else:
            print("NGMSA-CC: I'm sorry, I didn't understand your question.")
    else:
        print("NGMSA-CC: I'm sorry, I didn't understand your question.")

## Functionas Call

In [ ]:


if __name__ == "__main__":
    # Step 1: Replace 'your_document.docx' with the actual path to your Word document
    docx_file = "cc.docx"

    # Step 2: Read the Word document and extract table data
    table_data = read_table_from_docx(docx_file)

    # Step 3: Process the table data and generate chatbot responses
    chatbot_responses = generate_chatbot_responses(table_data)

    # Step 4: Chat with the bot
    chat_with_bot(chatbot_responses)


Chatbot: Hi! How Can I Assest You Today?.
You: cmc
NGMSA-CC: Did you mean: cms ?
You: yes
NGMSA-CC: Central monitoring site
You: what cms
NGMSA-CC: Central monitoring site
You: 3world
NGMSA-CC: Did you mean: 3d world ?
You: yes
NGMSA-CC: Set 3D models containing site topology including security devices (Access control, cameras, etc).


In summary, this code implements a simple chatbot that reads information from a Word document, processes user input, and generates responses based on pre-defined answers. It uses spaCy for NLP tasks like intent recognition and entity extraction and allows users to interact with the chatbot in a conversation-like manner.